In [ ]:
# Step 1: Suppress Warnings and Import Required Libraries
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

In [ ]:
# Step 2: Load Dataset
# Assuming you've downloaded 'GOOG.csv' from Yahoo Finance or Kaggle
# Make sure to place it in the same folder or provide full path
df = pd.read_csv("GOOG.csv")  # Google stock price dataset
df.head()

In [ ]:
# Step 3: Data Preprocessing
# We'll use 'Close' price for prediction
df = df[['Date', 'Close']]
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Visualize
df['Close'].plot(title="Google Stock Closing Price", figsize=(10, 5))
plt.grid(True)
plt.show()

In [ ]:
# Step 4: Prepare Time Series Data
data = df['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Create sequences
def create_sequences(data, seq_length=60):
    x, y = [], []
    for i in range(seq_length, len(data)):
        x.append(data[i - seq_length:i])
        y.append(data[i])
    return np.array(x), np.array(y)

sequence_length = 60
x, y = create_sequences(data_scaled, sequence_length)

# Train-test split (80-20)
split = int(0.8 * len(x))
x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]

print("Training shape:", x_train.shape, y_train.shape)
print("Testing shape:", x_test.shape, y_test.shape)

In [ ]:
# Step 5: Build the RNN Model
model = Sequential([
    SimpleRNN(50, activation='tanh', input_shape=(x_train.shape[1], 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
# Step 6: Train the Model
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
# Step 7: Model Evaluation
loss = model.evaluate(x_test, y_test)
print(f"\nTest Loss (MSE): {loss:.6f}")


In [ ]:
# Step 8: Predictions and Plotting
predicted = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted)
actual_prices = scaler.inverse_transform(y_test)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(actual_prices, label="Actual Price")
plt.plot(predicted_prices, label="Predicted Price")
plt.title("Google Stock Price Prediction using RNN")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Step 9: Predict Next Day's Stock Price
last_sequence = data_scaled[-sequence_length:]
next_input = last_sequence.reshape((1, sequence_length, 1))
next_pred = model.predict(next_input)
next_price = scaler.inverse_transform(next_pred)
print(f"Predicted next day's stock price: ${next_price[0][0]:.2f}")


In [ ]:
# Step 10: Predict Next 7 Days Stock Prices

future_predictions = []
input_seq = data_scaled[-sequence_length:]  # last 60 days

for _ in range(7):  # next 7 days
    pred = model.predict(input_seq.reshape(1, sequence_length, 1))
    future_predictions.append(pred[0][0])
    input_seq = np.append(input_seq, pred)[1:]  # slide window forward

# Inverse scale
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Print predicted values
for i, price in enumerate(future_predictions, 1):
    print(f"Day {i}: ${price[0]:.2f}")

In [ ]:
# Extend the plot with future predictions
last_actual_day = df.index[-1]
future_dates = pd.date_range(last_actual_day + pd.Timedelta(days=1), periods=7)

plt.figure(figsize=(10, 5))
plt.plot(df.index[-100:], df['Close'].values[-100:], label="Actual Price (last 100 days)")
plt.plot(future_dates, future_predictions, 'r--o', label="Predicted Next 7 Days")
plt.title("Google Stock Price Forecast")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.legend()
plt.grid(True)
plt.show()